In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import time
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.feature_selection import RFECV
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [2]:
train = pd.read_csv("/kaggle/input/samplednoinfnonnorm/RandomSamplingSMOTE_train_Sampled_NOINF_nonnorm.csv")
test = pd.read_csv("/kaggle/input/noinfnonnormtestt/X_test.csv")

In [3]:
X_fe_train = train.copy()
X_fe_train.head()

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Idle Mean,Idle Std,Idle Max,Idle Min,0,1,2,3,4,5
0,6,117986095,11,9,875,355,517,0,79.545455,154.043087,...,5.862518e+07,374022.010588,58889657,58360710,1.0,0.0,0.0,0.0,0.0,0.0
1,6,86160336,2,0,0,0,0,0,0.000000,0.000000,...,8.616034e+07,0.000000,86160336,86160336,1.0,0.0,0.0,0.0,0.0,0.0
2,6,3769550,14,8,1439,1731,725,0,102.785714,191.197111,...,0.000000e+00,0.000000,0,0,1.0,0.0,0.0,0.0,0.0,0.0
3,6,61337501,21,19,2727,9591,724,0,129.857143,242.631054,...,9.990232e+06,32318.260496,10003474,9924263,1.0,0.0,0.0,0.0,0.0,0.0
4,17,2573,1,1,30,207,30,30,30.000000,0.000000,...,0.000000e+00,0.000000,0,0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
X_fe_test = test.copy()
X_fe_test.head()

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Idle Mean,Idle Std,Idle Max,Idle Min,0,1,2,3,4,5
0,6,2471708,8,8,1148,1581,677,0,143.500000,228.129662,...,0.0,0.0,0,0,1,0,0,0,0,0
1,6,816,3,0,77,0,46,0,25.666667,23.459184,...,0.0,0.0,0,0,1,0,0,0,0,0
2,6,2495557,9,8,1789,1581,661,0,198.777778,270.943619,...,0.0,0.0,0,0,1,0,0,0,0,0
3,6,116,2,1,38,0,38,0,19.000000,26.870058,...,0.0,0.0,0,0,1,0,0,0,0,0
4,6,3958451,14,8,1441,1731,725,0,102.928571,191.148941,...,0.0,0.0,0,0,1,0,0,0,0,0


In [5]:
last_6_columns = X_fe_train.iloc[:, -6:]
Y_fe_train = pd.DataFrame(last_6_columns)
Y_fe_train.head()

,0,1,2,3,4,5
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
Y_fe_train.dtypes

0    float64
1    float64
2    float64
3    float64
4    float64
5    float64
dtype: object

In [7]:
last_6_columns_test = X_fe_test.iloc[:, -6:]
Y_fe_test = pd.DataFrame(last_6_columns_test)
Y_fe_test.head()

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0


In [8]:
X_fe_train = X_fe_train.drop(X_fe_train.columns[-6:], axis=1)
X_fe_train.head()

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,6,117986095,11,9,875,355,517,0,79.545455,154.043087,...,6,20,289957.000000,189894.940301,424233,155681,5.862518e+07,374022.010588,58889657,58360710
1,6,86160336,2,0,0,0,0,0,0.000000,0.000000,...,0,20,0.000000,0.000000,0,0,8.616034e+07,0.000000,86160336,86160336
2,6,3769550,14,8,1439,1731,725,0,102.785714,191.197111,...,7,20,0.000000,0.000000,0,0,0.000000e+00,0.000000,0,0
3,6,61337501,21,19,2727,9591,724,0,129.857143,242.631054,...,12,20,195588.333333,449991.678643,1114130,11845,9.990232e+06,32318.260496,10003474,9924263
4,17,2573,1,1,30,207,30,30,30.000000,0.000000,...,0,8,0.000000,0.000000,0,0,0.000000e+00,0.000000,0,0


In [9]:
X_fe_test = X_fe_test.drop(X_fe_test.columns[-6:], axis=1)
X_fe_test.head()

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,6,2471708,8,8,1148,1581,677,0,143.500000,228.129662,...,5,20,0.0,0.0,0,0,0.0,0.0,0,0
1,6,816,3,0,77,0,46,0,25.666667,23.459184,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
2,6,2495557,9,8,1789,1581,661,0,198.777778,270.943619,...,6,20,0.0,0.0,0,0,0.0,0.0,0,0
3,6,116,2,1,38,0,38,0,19.000000,26.870058,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
4,6,3958451,14,8,1441,1731,725,0,102.928571,191.148941,...,7,20,0.0,0.0,0,0,0.0,0.0,0,0


In [10]:
X_fe_test.dtypes

Protocol             int64
Flow Duration        int64
Tot Fwd Pkts         int64
Tot Bwd Pkts         int64
TotLen Fwd Pkts      int64
                    ...   
Active Min           int64
Idle Mean          float64
Idle Std           float64
Idle Max             int64
Idle Min             int64
Length: 69, dtype: object

# RFE using Adaboost

In [11]:
base_estimator = DecisionTreeClassifier(max_depth=5)
ada_boost_model = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=100, random_state=0)
Y_fe_train_1d = np.argmax(Y_fe_train.values, axis=1)
sel = SelectFromModel(ada_boost_model)
sel.fit(X_fe_train, Y_fe_train_1d)
features = X_fe_train.columns[sel.get_support()]
print("Selected features:", features)
print("Number of selected features:", len(features))
X_train_rfe = sel.transform(X_fe_train)
X_test_rfe = sel.transform(X_fe_test)

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Selected features: Index(['Flow Duration', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Min', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Min',
       'Bwd IAT Min', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Init Fwd Win Byts',
       'Init Bwd Win Byts', 'Fwd Seg Size Min'],
      dtype='object')
Number of selected features: 14


In [12]:
Y_fe_train_1d = np.argmax(Y_fe_train.values, axis=1)

random_state = 42
clf_rfe_keys = ["AdaBoost"]
clf_rfe_values = [AdaBoostClassifier(random_state=random_state)]

unique_classes = np.unique(Y_fe_train_1d)
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
Y_fe_train_1d_mapped = np.vectorize(class_mapping.get)(Y_fe_train_1d)

clf_rfe_keys += ["Random Forest", "XGBoost", "CatBoost","Decision Tree",]
clf_rfe_values += [RandomForestClassifier(random_state=random_state), 
                    XGBClassifier(random_state=random_state), 
                    CatBoostClassifier(random_state=random_state, verbose=0),DecisionTreeClassifier(random_state=random_state)]

clfs_rfe = dict(zip(clf_rfe_keys, clf_rfe_values))

print("Model training using feature-selected data: started!")
for clf_rfe_name, clf_rfe in clfs_rfe.items():
    if clf_rfe_name == 'XGBoost':
        clf_rfe.fit(X_train_rfe, Y_fe_train_1d_mapped)
    else:
        clf_rfe.fit(X_train_rfe, Y_fe_train_1d)
    clfs_rfe[clf_rfe_name] = clf_rfe
    print(clf_rfe_name, "training: done!", )
print("Model training using feature-selected data: done!")

Model training using feature-selected data: started!
AdaBoost training: done!
Random Forest training: done!
XGBoost training: done!
CatBoost training: done!
Decision Tree training: done!
Model training using feature-selected data: done!


In [13]:
Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)

acc_rfe = []
f1_rfe = []
precision_rfe = []
recall_rfe = []

max_f1 = {"score": 0, "model": ""}
max_precision = {"score": 0, "model": ""}
max_recall = {"score": 0, "model": ""}

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_test_rfe)
    
    f1 = f1_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    precision = precision_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    recall = recall_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    
    f1_rfe.append(f1)
    precision_rfe.append(precision)
    recall_rfe.append(recall)
    
    # Update maximum metrics and corresponding model names
    if f1 > max_f1["score"]:
        max_f1 = {"score": f1, "model": clf_rfe_name}
    if precision > max_precision["score"]:
        max_precision = {"score": precision, "model": clf_rfe_name}
    if recall > max_recall["score"]:
        max_recall = {"score": recall, "model": clf_rfe_name}    
    
    
acc_all = pd.DataFrame({"Feature-selected dataset": acc_rfe},index=clf_rfe_keys)

metrics_all = pd.DataFrame({
    "Feature-selected dataset - Accuracy": acc_rfe,
    "Feature-selected dataset - F1 Score": f1_rfe,
    "Feature-selected dataset - Precision": precision_rfe,
    "Feature-selected dataset - Recall": recall_rfe,
}, index=clf_rfe_keys)

metrics_all

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Feature-selected dataset - Accuracy,Feature-selected dataset - F1 Score,Feature-selected dataset - Precision,Feature-selected dataset - Recall
AdaBoost,0.685033,0.135841,0.157970,0.267785
Random Forest,0.998755,0.755121,0.718276,0.878706
XGBoost,0.998818,0.736825,0.696229,0.881048
CatBoost,0.998027,0.684502,0.616459,0.925514
Decision Tree,0.999323,0.731612,0.685656,0.840564


In [14]:
# Print maximum metrics and corresponding model names
max_acc_rfe = acc_all["Feature-selected dataset"].max()
best_model_rfe = acc_all[acc_all["Feature-selected dataset"] == max_acc_rfe].index[0]
print("Max Accuracy:", max_acc_rfe, "achieved by:", best_model_rfe)
print("Max F1 score: ", max_f1["score"], " achieved by ", max_f1["model"])
print("Max Precision: ", max_precision["score"], " achieved by ", max_precision["model"])
print("Max Recall: ", max_recall["score"], " achieved by ", max_recall["model"])

Max Accuracy: 0.99932292672485 achieved by: Decision Tree
Max F1 score:  0.7551212191327679  achieved by  Random Forest
Max Precision:  0.7182760242991626  achieved by  Random Forest
Max Recall:  0.9255137351851978  achieved by  CatBoost


In [15]:
def calculate_accuracy_per_class(y_true, y_pred):
    classes = set(y_true)
    accuracy_per_class = {}
    for c in classes:
        samples = [y for y, pred in zip(y_true, y_pred) if y == c]
        correctly_classified = [y for y, pred in zip(y_true, y_pred) if y == c and y == pred]
        accuracy_per_class[c] = len(correctly_classified) / len(samples) if samples else 0
    return accuracy_per_class

Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)
acc_rfe = []
acc_per_class_rfe = []

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_test_rfe)
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    accuracy_per_class = calculate_accuracy_per_class(Y_fe_test_1d, y_rfe_pred)
    acc_per_class_rfe.append(accuracy_per_class)

acc_all = pd.DataFrame({"Feature-selected dataset": acc_per_class_rfe},index=clf_rfe_keys)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
print(acc_all)

                                                                                                                  Feature-selected dataset
AdaBoost                    {0: 0.7166337631482216, 1: 0.0, 2: 0.4782608695652174, 3: 0.0, 4: 0.4117647058823529, 5: 5.31632110579479e-05}
Random Forest  {0: 0.9987658137717657, 1: 0.8918918918918919, 2: 0.9347826086956522, 3: 0.8, 4: 0.6470588235294118, 5: 0.9997341839447103}
XGBoost         {0: 0.9988315396655769, 1: 0.8468468468468469, 2: 0.9347826086956522, 3: 0.8, 4: 0.7058823529411765, 5: 0.999946836788942}
CatBoost       {0: 0.9980038802620272, 1: 0.8558558558558559, 2: 0.9347826086956522, 3: 1.0, 4: 0.7647058823529411, 5: 0.9997341839447103}
Decision Tree  {0: 0.9993914269091546, 1: 0.8018018018018018, 2: 0.9130434782608695, 3: 0.8, 4: 0.5294117647058824, 5: 0.9997341839447103}


In [16]:
max_accuracy_per_class = {}
for accuracy_dict in acc_per_class_rfe:
    for c, accuracy in accuracy_dict.items():
        if c not in max_accuracy_per_class or accuracy > max_accuracy_per_class[c]:
            max_accuracy_per_class[c] = accuracy

print(max_accuracy_per_class)

{0: 0.9993914269091546, 1: 0.8918918918918919, 2: 0.9347826086956522, 3: 1.0, 4: 0.7647058823529411, 5: 0.999946836788942}


# RFE using XGB

In [11]:
Y_fe_train_1d = np.argmax(Y_fe_train.values, axis=1)
unique_classes = np.unique(Y_fe_train_1d)
class_mapping = {class_val: idx for idx, class_val in enumerate(unique_classes)}
Y_fe_train_mapped = np.vectorize(class_mapping.get)(Y_fe_train_1d)
xgb = XGBClassifier(n_estimators=100, random_state=0, n_jobs=-1)
xgb.fit(X_fe_train, Y_fe_train_mapped)
sel = SelectFromModel(xgb)
sel.fit(X_fe_train, Y_fe_train_mapped)
features = X_fe_train.columns[sel.get_support()]
print("Selected features:", features)
print("Number of selected features:", len(features))
X_train_rfe = sel.transform(X_fe_train)
X_test_rfe = sel.transform(X_fe_test)

Selected features: Index(['Protocol', 'Tot Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Mean',
       'Fwd Pkt Len Std', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std',
       'Flow IAT Max', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max',
       'Pkt Len Std', 'RST Flag Cnt', 'ECE Flag Cnt', 'Fwd Act Data Pkts',
       'Fwd Seg Size Min'],
      dtype='object')
Number of selected features: 16


In [12]:
Y_fe_train_1d = np.argmax(Y_fe_train.values, axis=1)

random_state = 42
clf_rfe_keys = ["AdaBoost"]
clf_rfe_values = [AdaBoostClassifier(random_state=random_state)]

unique_classes = np.unique(Y_fe_train_1d)
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
Y_fe_train_1d_mapped = np.vectorize(class_mapping.get)(Y_fe_train_1d)

clf_rfe_keys += ["Random Forest", "XGBoost", "CatBoost","Decision Tree",]
clf_rfe_values += [RandomForestClassifier(random_state=random_state), 
                    XGBClassifier(random_state=random_state), 
                    CatBoostClassifier(random_state=random_state, verbose=0),DecisionTreeClassifier(random_state=random_state)]

clfs_rfe = dict(zip(clf_rfe_keys, clf_rfe_values))

print("Model training using feature-selected data: started!")
for clf_rfe_name, clf_rfe in clfs_rfe.items():
    if clf_rfe_name == 'XGBoost':
        clf_rfe.fit(X_train_rfe, Y_fe_train_1d_mapped)
    else:
        clf_rfe.fit(X_train_rfe, Y_fe_train_1d)
    clfs_rfe[clf_rfe_name] = clf_rfe
    print(clf_rfe_name, "training: done!", )
print("Model training using feature-selected data: done!")

Model training using feature-selected data: started!
AdaBoost training: done!
Random Forest training: done!
XGBoost training: done!
CatBoost training: done!
Decision Tree training: done!
Model training using feature-selected data: done!


In [14]:
Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)

acc_rfe = []
f1_rfe = []
precision_rfe = []
recall_rfe = []

max_f1 = {"score": 0, "model": ""}
max_precision = {"score": 0, "model": ""}
max_recall = {"score": 0, "model": ""}

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_test_rfe)
    
    f1 = f1_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    precision = precision_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    recall = recall_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    
    f1_rfe.append(f1)
    precision_rfe.append(precision)
    recall_rfe.append(recall)
    
    # Update maximum metrics and corresponding model names
    if f1 > max_f1["score"]:
        max_f1 = {"score": f1, "model": clf_rfe_name}
    if precision > max_precision["score"]:
        max_precision = {"score": precision, "model": clf_rfe_name}
    if recall > max_recall["score"]:
        max_recall = {"score": recall, "model": clf_rfe_name}    
    
    
acc_all = pd.DataFrame({"Feature-selected dataset": acc_rfe},index=clf_rfe_keys)

metrics_all = pd.DataFrame({
    "Feature-selected dataset - Accuracy": acc_rfe,
    "Feature-selected dataset - F1 Score": f1_rfe,
    "Feature-selected dataset - Precision": precision_rfe,
    "Feature-selected dataset - Recall": recall_rfe,
}, index=clf_rfe_keys)

metrics_all

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Feature-selected dataset - Accuracy,Feature-selected dataset - F1 Score,Feature-selected dataset - Precision,Feature-selected dataset - Recall
AdaBoost,0.535672,0.116785,0.154276,0.260348
Random Forest,0.984942,0.462043,0.449116,0.839194
XGBoost,0.985298,0.477718,0.451169,0.880270
CatBoost,0.985097,0.491180,0.462803,0.905665
Decision Tree,0.985267,0.462196,0.449198,0.830931


In [15]:
# Print maximum metrics and corresponding model names
max_acc_rfe = acc_all["Feature-selected dataset"].max()
best_model_rfe = acc_all[acc_all["Feature-selected dataset"] == max_acc_rfe].index[0]
print("Max Accuracy:", max_acc_rfe, "achieved by:", best_model_rfe)
print("Max F1 score: ", max_f1["score"], " achieved by ", max_f1["model"])
print("Max Precision: ", max_precision["score"], " achieved by ", max_precision["model"])
print("Max Recall: ", max_recall["score"], " achieved by ", max_recall["model"])

Max Accuracy: 0.985297505066416 achieved by: XGBoost
Max F1 score:  0.4911803937112049  achieved by  CatBoost
Max Precision:  0.4628030143751125  achieved by  CatBoost
Max Recall:  0.9056653792365882  achieved by  CatBoost


In [16]:
def calculate_accuracy_per_class(y_true, y_pred):
    classes = set(y_true)
    accuracy_per_class = {}
    for c in classes:
        samples = [y for y, pred in zip(y_true, y_pred) if y == c]
        correctly_classified = [y for y, pred in zip(y_true, y_pred) if y == c and y == pred]
        accuracy_per_class[c] = len(correctly_classified) / len(samples) if samples else 0
    return accuracy_per_class

Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)
acc_rfe = []
acc_per_class_rfe = []

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_test_rfe)
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    accuracy_per_class = calculate_accuracy_per_class(Y_fe_test_1d, y_rfe_pred)
    acc_per_class_rfe.append(accuracy_per_class)

acc_all = pd.DataFrame({"Feature-selected dataset": acc_per_class_rfe},index=clf_rfe_keys)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
print(acc_all)

                                                                                                                  Feature-selected dataset
AdaBoost                   {0: 0.5602548217245987, 1: 0.4864864864864865, 2: 0.45652173913043476, 3: 0.0, 4: 0.058823529411764705, 5: 0.0}
Random Forest  {0: 0.9843718430270912, 1: 0.7387387387387387, 2: 0.8478260869565217, 3: 0.7, 4: 0.7647058823529411, 5: 0.9995215311004785}
XGBoost        {0: 0.9847369868815985, 1: 0.7477477477477478, 2: 0.8260869565217391, 3: 0.9, 4: 0.8235294117647058, 5: 0.9995215311004785}
CatBoost       {0: 0.9845130319841674, 1: 0.7567567567567568, 2: 0.8695652173913043, 3: 1.0, 4: 0.8235294117647058, 5: 0.9996278575225943}
Decision Tree  {0: 0.9847175125426914, 1: 0.7477477477477478, 2: 0.8478260869565217, 3: 0.7, 4: 0.7058823529411765, 5: 0.9994152046783625}


# RFE using CatBoost

In [17]:
from catboost import CatBoostClassifier, Pool

Y_fe_train_1d = np.argmax(Y_fe_train.values, axis=1)

unique_classes = np.unique(Y_fe_train_1d)
class_mapping = {class_val: idx for idx, class_val in enumerate(unique_classes)}
Y_fe_train_mapped = np.vectorize(class_mapping.get)(Y_fe_train_1d)
catboost = CatBoostClassifier(iterations=100, random_state=0, thread_count=-1)
catboost.fit(X_fe_train, Y_fe_train_mapped)
feature_importances = catboost.get_feature_importance(data=Pool(X_fe_train, label=Y_fe_train_mapped))
threshold = np.percentile(feature_importances, 90)  # Adjust the percentile as needed
selected_features = np.array(X_fe_train.columns)[feature_importances >= threshold]
selected_features_test = np.array(X_fe_test.columns)[feature_importances >= threshold]
print("Selected features:", selected_features)
print("Number of selected features:", len(selected_features))
X_train_rfe = sel.transform(X_fe_train)
X_test_rfe = sel.transform(X_fe_test)

Learning rate set to 0.5
0:	learn: 0.5650722	total: 2.21s	remaining: 3m 39s
1:	learn: 0.3257572	total: 4.78s	remaining: 3m 54s
2:	learn: 0.2239370	total: 7.95s	remaining: 4m 16s
3:	learn: 0.1734388	total: 10.2s	remaining: 4m 4s
4:	learn: 0.1405918	total: 12.8s	remaining: 4m 2s
5:	learn: 0.1190447	total: 14.7s	remaining: 3m 50s
6:	learn: 0.1030439	total: 17.6s	remaining: 3m 53s
7:	learn: 0.0915503	total: 19.4s	remaining: 3m 43s
8:	learn: 0.0773834	total: 21.7s	remaining: 3m 39s
9:	learn: 0.0673959	total: 24.1s	remaining: 3m 36s
10:	learn: 0.0644308	total: 25.6s	remaining: 3m 26s
11:	learn: 0.0597591	total: 27.3s	remaining: 3m 19s
12:	learn: 0.0536391	total: 29.7s	remaining: 3m 18s
13:	learn: 0.0475354	total: 31.6s	remaining: 3m 14s
14:	learn: 0.0439314	total: 33.8s	remaining: 3m 11s
15:	learn: 0.0402602	total: 35.9s	remaining: 3m 8s
16:	learn: 0.0386048	total: 37.5s	remaining: 3m 3s
17:	learn: 0.0376231	total: 38.9s	remaining: 2m 57s
18:	learn: 0.0357661	total: 40.8s	remaining: 2m 53s
1

In [18]:
Y_fe_train_1d = np.argmax(Y_fe_train.values, axis=1)

random_state = 42
clf_rfe_keys = ["AdaBoost"]
clf_rfe_values = [AdaBoostClassifier(random_state=random_state)]

unique_classes = np.unique(Y_fe_train_1d)
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
Y_fe_train_1d_mapped = np.vectorize(class_mapping.get)(Y_fe_train_1d)

clf_rfe_keys += ["Random Forest", "XGBoost", "CatBoost","Decision Tree",]
clf_rfe_values += [RandomForestClassifier(random_state=random_state), 
                    XGBClassifier(random_state=random_state), 
                    CatBoostClassifier(random_state=random_state, verbose=0),DecisionTreeClassifier(random_state=random_state)]

clfs_rfe = dict(zip(clf_rfe_keys, clf_rfe_values))

print("Model training using feature-selected data: started!")
for clf_rfe_name, clf_rfe in clfs_rfe.items():
    if clf_rfe_name == 'XGBoost':
        clf_rfe.fit(X_train_rfe, Y_fe_train_1d_mapped)
    else:
        clf_rfe.fit(X_train_rfe, Y_fe_train_1d)
    clfs_rfe[clf_rfe_name] = clf_rfe
    print(clf_rfe_name, "training: done!", )
print("Model training using feature-selected data: done!")

Model training using feature-selected data: started!
AdaBoost training: done!
Random Forest training: done!
XGBoost training: done!
CatBoost training: done!
Decision Tree training: done!
Model training using feature-selected data: done!


In [19]:
Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)

acc_rfe = []
f1_rfe = []
precision_rfe = []
recall_rfe = []

max_f1 = {"score": 0, "model": ""}
max_precision = {"score": 0, "model": ""}
max_recall = {"score": 0, "model": ""}

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_test_rfe)
    
    f1 = f1_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    precision = precision_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    recall = recall_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    
    f1_rfe.append(f1)
    precision_rfe.append(precision)
    recall_rfe.append(recall)
    
    # Update maximum metrics and corresponding model names
    if f1 > max_f1["score"]:
        max_f1 = {"score": f1, "model": clf_rfe_name}
    if precision > max_precision["score"]:
        max_precision = {"score": precision, "model": clf_rfe_name}
    if recall > max_recall["score"]:
        max_recall = {"score": recall, "model": clf_rfe_name}    
    
    
acc_all = pd.DataFrame({"Feature-selected dataset": acc_rfe},index=clf_rfe_keys)

metrics_all = pd.DataFrame({
    "Feature-selected dataset - Accuracy": acc_rfe,
    "Feature-selected dataset - F1 Score": f1_rfe,
    "Feature-selected dataset - Precision": precision_rfe,
    "Feature-selected dataset - Recall": recall_rfe,
}, index=clf_rfe_keys)

metrics_all

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Feature-selected dataset - Accuracy,Feature-selected dataset - F1 Score,Feature-selected dataset - Precision,Feature-selected dataset - Recall
AdaBoost,0.535672,0.116785,0.154276,0.260348
Random Forest,0.984942,0.462043,0.449116,0.839194
XGBoost,0.985298,0.477718,0.451169,0.880270
CatBoost,0.985097,0.491180,0.462803,0.905665
Decision Tree,0.985267,0.462196,0.449198,0.830931


In [21]:
# Print maximum metrics and corresponding model names
max_acc_rfe = acc_all["Feature-selected dataset"].max()
best_model_rfe = acc_all[acc_all["Feature-selected dataset"] == max_acc_rfe].index[0]
print("Max Accuracy:", max_acc_rfe, "achieved by:", best_model_rfe)
print("Max F1 score: ", max_f1["score"], " achieved by ", max_f1["model"])
print("Max Precision: ", max_precision["score"], " achieved by ", max_precision["model"])
print("Max Recall: ", max_recall["score"], " achieved by ", max_recall["model"])

Max Accuracy: 0.985297505066416 achieved by: XGBoost
Max F1 score:  0.4911803937112049  achieved by  CatBoost
Max Precision:  0.4628030143751125  achieved by  CatBoost
Max Recall:  0.9056653792365882  achieved by  CatBoost


In [23]:
def calculate_accuracy_per_class(y_true, y_pred):
    classes = set(y_true)
    accuracy_per_class = {}
    for c in classes:
        samples = [y for y, pred in zip(y_true, y_pred) if y == c]
        correctly_classified = [y for y, pred in zip(y_true, y_pred) if y == c and y == pred]
        accuracy_per_class[c] = len(correctly_classified) / len(samples) if samples else 0
    return accuracy_per_class

Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)
acc_rfe = []
acc_per_class_rfe = []

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_test_rfe)
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    accuracy_per_class = calculate_accuracy_per_class(Y_fe_test_1d, y_rfe_pred)
    acc_per_class_rfe.append(accuracy_per_class)

acc_all = pd.DataFrame({"Feature-selected dataset": acc_per_class_rfe},index=clf_rfe_keys)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
print(acc_all)

                                                                                                                  Feature-selected dataset
AdaBoost                   {0: 0.5602548217245987, 1: 0.4864864864864865, 2: 0.45652173913043476, 3: 0.0, 4: 0.058823529411764705, 5: 0.0}
Random Forest  {0: 0.9843718430270912, 1: 0.7387387387387387, 2: 0.8478260869565217, 3: 0.7, 4: 0.7647058823529411, 5: 0.9995215311004785}
XGBoost        {0: 0.9847369868815985, 1: 0.7477477477477478, 2: 0.8260869565217391, 3: 0.9, 4: 0.8235294117647058, 5: 0.9995215311004785}
CatBoost       {0: 0.9845130319841674, 1: 0.7567567567567568, 2: 0.8695652173913043, 3: 1.0, 4: 0.8235294117647058, 5: 0.9996278575225943}
Decision Tree  {0: 0.9847175125426914, 1: 0.7477477477477478, 2: 0.8478260869565217, 3: 0.7, 4: 0.7058823529411765, 5: 0.9994152046783625}


# RFE using DT

In [24]:
decision_tree = DecisionTreeClassifier(random_state=42, max_depth=5)
Y_fe_train_1d = np.argmax(Y_fe_train.values, axis=1)
sel = SelectFromModel(decision_tree)
sel.fit(X_fe_train, Y_fe_train_1d)
features = X_fe_train.columns[sel.get_support()]
print("Selected features:", features)
print("Number of selected features:", len(features))
X_train_rfe = sel.transform(X_fe_train)
X_test_rfe = sel.transform(X_fe_test)

Selected features: Index(['Tot Bwd Pkts', 'Bwd Pkt Len Max', 'Bwd Pkt Len Mean', 'Flow IAT Max',
       'Bwd Pkts/s', 'Pkt Len Min', 'Fwd Seg Size Min'],
      dtype='object')
Number of selected features: 7


In [26]:
Y_fe_train_1d = np.argmax(Y_fe_train.values, axis=1)
training_times = []

random_state = 42
clf_rfe_keys = ["AdaBoost"]
clf_rfe_values = [AdaBoostClassifier(random_state=random_state)]

unique_classes = np.unique(Y_fe_train_1d)
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
Y_fe_train_1d_mapped = np.vectorize(class_mapping.get)(Y_fe_train_1d)

clf_rfe_keys += ["Random Forest", "XGBoost", "CatBoost","Decision Tree",]
clf_rfe_values += [RandomForestClassifier(random_state=random_state), 
                    XGBClassifier(random_state=random_state), 
                    CatBoostClassifier(random_state=random_state, verbose=0),DecisionTreeClassifier(random_state=random_state)]

clfs_rfe = dict(zip(clf_rfe_keys, clf_rfe_values))

print("Model training using feature-selected data: started!")
for clf_rfe_name, clf_rfe in clfs_rfe.items():
    start_time = time.time()
    if clf_rfe_name == 'XGBoost':
        clf_rfe.fit(X_train_rfe, Y_fe_train_1d_mapped)
    else:
        clf_rfe.fit(X_train_rfe, Y_fe_train_1d)
    clfs_rfe[clf_rfe_name] = clf_rfe
    end_time = time.time()
    training_time = end_time - start_time
    training_times.append(training_time)
    print(clf_rfe_name, "training: done!", )
print("Model training using feature-selected data: done!")

Model training using feature-selected data: started!
AdaBoost training: done!
Random Forest training: done!
XGBoost training: done!
CatBoost training: done!
Decision Tree training: done!
Model training using feature-selected data: done!


In [27]:
Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)

acc_rfe = []
f1_rfe = []
precision_rfe = []
recall_rfe = []

max_f1 = {"score": 0, "model": ""}
max_precision = {"score": 0, "model": ""}
max_recall = {"score": 0, "model": ""}

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_test_rfe)
    
    f1 = f1_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    precision = precision_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    recall = recall_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    
    f1_rfe.append(f1)
    precision_rfe.append(precision)
    recall_rfe.append(recall)
    
    # Update maximum metrics and corresponding model names
    if f1 > max_f1["score"]:
        max_f1 = {"score": f1, "model": clf_rfe_name}
    if precision > max_precision["score"]:
        max_precision = {"score": precision, "model": clf_rfe_name}
    if recall > max_recall["score"]:
        max_recall = {"score": recall, "model": clf_rfe_name}    
    
    
acc_all = pd.DataFrame({"Feature-selected dataset": acc_rfe},index=clf_rfe_keys)

metrics_all = pd.DataFrame({
    "Feature-selected dataset - Accuracy": acc_rfe,
    "Feature-selected dataset - F1 Score": f1_rfe,
    "Feature-selected dataset - Precision": precision_rfe,
    "Feature-selected dataset - Recall": recall_rfe,
}, index=clf_rfe_keys)

metrics_all

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Feature-selected dataset - Accuracy,Feature-selected dataset - F1 Score,Feature-selected dataset - Precision,Feature-selected dataset - Recall
AdaBoost,0.891263,0.322900,0.328476,0.409744
Random Forest,0.972196,0.454516,0.445747,0.819906
XGBoost,0.974250,0.470748,0.447992,0.864029
CatBoost,0.971679,0.483944,0.459638,0.906504
Decision Tree,0.972796,0.454563,0.445742,0.814886


In [29]:
# Print maximum metrics and corresponding model names
max_acc_rfe = acc_all["Feature-selected dataset"].max()
best_model_rfe = acc_all[acc_all["Feature-selected dataset"] == max_acc_rfe].index[0]
print("Max Accuracy:", max_acc_rfe, "achieved by:", best_model_rfe)
print("Max F1 score: ", max_f1["score"], " achieved by ", max_f1["model"])
print("Max Precision: ", max_precision["score"], " achieved by ", max_precision["model"])
print("Max Recall: ", max_recall["score"], " achieved by ", max_recall["model"])

Max Accuracy: 0.9742502751337278 achieved by: XGBoost
Max F1 score:  0.4839443219349042  achieved by  CatBoost
Max Precision:  0.45963838130136464  achieved by  CatBoost
Max Recall:  0.9065041601505999  achieved by  CatBoost


In [30]:
def calculate_accuracy_per_class(y_true, y_pred):
    classes = set(y_true)
    accuracy_per_class = {}
    for c in classes:
        samples = [y for y, pred in zip(y_true, y_pred) if y == c]
        correctly_classified = [y for y, pred in zip(y_true, y_pred) if y == c and y == pred]
        accuracy_per_class[c] = len(correctly_classified) / len(samples) if samples else 0
    return accuracy_per_class

Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)
acc_rfe = []
acc_per_class_rfe = []

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_test_rfe)
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    accuracy_per_class = calculate_accuracy_per_class(Y_fe_test_1d, y_rfe_pred)
    acc_per_class_rfe.append(accuracy_per_class)

acc_all = pd.DataFrame({"Feature-selected dataset": acc_per_class_rfe},index=clf_rfe_keys)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
print(acc_all)

                                                                                                                  Feature-selected dataset
AdaBoost                    {0: 0.8867396792089524, 1: 0.0, 2: 0.45652173913043476, 3: 0.0, 4: 0.11764705882352941, 5: 0.9975544922913344}
Random Forest  {0: 0.9710489609223046, 1: 0.7387387387387387, 2: 0.8043478260869565, 3: 0.7, 4: 0.7058823529411765, 5: 0.9994152046783625}
XGBoost        {0: 0.9731911382020804, 1: 0.7207207207207207, 2: 0.8260869565217391, 3: 0.9, 4: 0.7647058823529411, 5: 0.9994683678894205}
CatBoost       {0: 0.9704793365092734, 1: 0.7387387387387387, 2: 0.8478260869565217, 3: 1.0, 4: 0.8823529411764706, 5: 0.9996278575225943}
Decision Tree   {0: 0.9716818769367839, 1: 0.7297297297297297, 2: 0.782608695652174, 3: 0.7, 4: 0.7058823529411765, 5: 0.9994152046783625}


# Original  Dataset

In [31]:
Y_fe_train_1d = np.argmax(Y_fe_train.values, axis=1)
training_times = []

random_state = 42
clf_rfe_keys = ["AdaBoost"]
clf_rfe_values = [AdaBoostClassifier(random_state=random_state)]

unique_classes = np.unique(Y_fe_train_1d)
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
Y_fe_train_1d_mapped = np.vectorize(class_mapping.get)(Y_fe_train_1d)

clf_rfe_keys += ["Random Forest", "XGBoost", "CatBoost","Decision Tree",]
clf_rfe_values += [RandomForestClassifier(random_state=random_state), 
                    XGBClassifier(random_state=random_state), 
                    CatBoostClassifier(random_state=random_state, verbose=0),DecisionTreeClassifier(random_state=random_state)]

clfs_rfe = dict(zip(clf_rfe_keys, clf_rfe_values))

print("Model training using feature-selected data: started!")
for clf_rfe_name, clf_rfe in clfs_rfe.items():
    start_time = time.time()
    if clf_rfe_name == 'XGBoost':
        clf_rfe.fit(X_fe_train, Y_fe_train_1d_mapped)
    else:
        clf_rfe.fit(X_fe_train, Y_fe_train_1d)
    clfs_rfe[clf_rfe_name] = clf_rfe
    end_time = time.time()
    training_time = end_time - start_time
    training_times.append(training_time)
    print(clf_rfe_name, "training: done!", )
print("Model training using feature-selected data: done!")

Model training using feature-selected data: started!
AdaBoost training: done!
Random Forest training: done!
XGBoost training: done!
CatBoost training: done!
Decision Tree training: done!
Model training using feature-selected data: done!


In [32]:
Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)

acc_rfe = []
f1_rfe = []
precision_rfe = []
recall_rfe = []

max_f1 = {"score": 0, "model": ""}
max_precision = {"score": 0, "model": ""}
max_recall = {"score": 0, "model": ""}

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_fe_test)
    
    f1 = f1_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    precision = precision_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    recall = recall_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    
    f1_rfe.append(f1)
    precision_rfe.append(precision)
    recall_rfe.append(recall)
    
    # Update maximum metrics and corresponding model names
    if f1 > max_f1["score"]:
        max_f1 = {"score": f1, "model": clf_rfe_name}
    if precision > max_precision["score"]:
        max_precision = {"score": precision, "model": clf_rfe_name}
    if recall > max_recall["score"]:
        max_recall = {"score": recall, "model": clf_rfe_name}    
    
    
acc_all = pd.DataFrame({"Feature-selected dataset": acc_rfe},index=clf_rfe_keys)

metrics_all = pd.DataFrame({
    "Feature-selected dataset - Accuracy": acc_rfe,
    "Feature-selected dataset - F1 Score": f1_rfe,
    "Feature-selected dataset - Precision": precision_rfe,
    "Feature-selected dataset - Recall": recall_rfe,
}, index=clf_rfe_keys)

metrics_all

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Feature-selected dataset - Accuracy,Feature-selected dataset - F1 Score,Feature-selected dataset - Precision,Feature-selected dataset - Recall
AdaBoost,0.696471,0.137376,0.157045,0.320590
Random Forest,0.999330,0.786631,0.749810,0.869976
XGBoost,0.999165,0.764462,0.724688,0.884103
CatBoost,0.998892,0.713330,0.644964,0.916482
Decision Tree,0.999488,0.768166,0.723679,0.860040


In [33]:
# Print maximum metrics and corresponding model names
max_acc_rfe = acc_all["Feature-selected dataset"].max()
best_model_rfe = acc_all[acc_all["Feature-selected dataset"] == max_acc_rfe].index[0]
print("Max Accuracy:", max_acc_rfe, "achieved by:", best_model_rfe)
print("Max F1 score: ", max_f1["score"], " achieved by ", max_f1["model"])
print("Max Precision: ", max_precision["score"], " achieved by ", max_precision["model"])
print("Max Recall: ", max_recall["score"], " achieved by ", max_recall["model"])

Max Accuracy: 0.9994881232971374 achieved by: Decision Tree
Max F1 score:  0.7866308371753372  achieved by  Random Forest
Max Precision:  0.7498104796185684  achieved by  Random Forest
Max Recall:  0.9164817365766819  achieved by  CatBoost


In [34]:
def calculate_accuracy_per_class(y_true, y_pred):
    classes = set(y_true)
    accuracy_per_class = {}
    for c in classes:
        samples = [y for y, pred in zip(y_true, y_pred) if y == c]
        correctly_classified = [y for y, pred in zip(y_true, y_pred) if y == c and y == pred]
        accuracy_per_class[c] = len(correctly_classified) / len(samples) if samples else 0
    return accuracy_per_class

Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)
acc_rfe = []
acc_per_class_rfe = []

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_fe_test)
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    accuracy_per_class = calculate_accuracy_per_class(Y_fe_test_1d, y_rfe_pred)
    acc_per_class_rfe.append(accuracy_per_class)

acc_all = pd.DataFrame({"Feature-selected dataset": acc_per_class_rfe},index=clf_rfe_keys)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
print(acc_all)

                                                                                                                  Feature-selected dataset
AdaBoost                       {0: 0.72853258422043, 1: 0.0990990990990991, 2: 0.9782608695652174, 3: 0.0, 4: 0.11764705882352941, 5: 0.0}
Random Forest  {0: 0.9993914269091546, 1: 0.8018018018018018, 2: 0.9130434782608695, 3: 0.8, 4: 0.7058823529411765, 5: 0.9997341839447103}
XGBoost        {0: 0.9991918149353574, 1: 0.8648648648648649, 2: 0.9347826086956522, 3: 0.8, 4: 0.7058823529411765, 5: 0.9998936735778841}
CatBoost       {0: 0.9989143056059319, 1: 0.8378378378378378, 2: 0.9565217391304348, 3: 1.0, 4: 0.7058823529411765, 5: 0.9997341839447103}
Decision Tree  {0: 0.9995447873280476, 1: 0.8378378378378378, 2: 0.9347826086956522, 3: 0.8, 4: 0.5882352941176471, 5: 0.9998405103668262}
